In [59]:
import cv2 as cv
import dlib
import numpy as np
import datetime
from IPython.display import clear_output

In [96]:
class PredictorBlinkClass:
    def __init__(self, threshold):
        self.detector = dlib.get_frontal_face_detector()
        self.predictor = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat')
        self.right_eye = (36, 42)
        self.left_eye = (42, 48)
        self.threshold = threshold
        self.blinks = 0
        self.crossed_right = False
        self.crossed_left = False
        self.time_crossed = 0
        self.blink_time = 1000
        self.prop = 1.15
    
    def rect_to_ndarray(self, shape, length):
        coords = np.zeros((length, 2), dtype="int")
        for i in range(0, length):
            coords[i] = (shape.part(i).x, shape.part(i).y)
        return coords

    def distance(self, p1, p2):
        temp = p1 - p2
        return np.sqrt(np.dot(temp.T, temp))

    def get_ear(self, eye):
        a = self.distance(eye[1], eye[5])
        b = self.distance(eye[2], eye[4])
        c = self.distance(eye[0], eye[3])
        return (a + b) / (2 * c)

    def set_threshold(self, threshold):
        self.threshold = threshold

    def set_blinks(self, blinks):
        self.blinks = blinks

    def predict(self, image_color, time, RGB=(0, 0, 255)):
        image_predicted = image_color.copy()
        image_gray = cv.cvtColor(image_color, cv.COLOR_BGR2GRAY)
        faceboxes = self.detector(image_gray, 1)
        ear_right = 0.0
        ear_left = 0.0
        
        for facebox in faceboxes:
            if self.time_crossed > 0 and time - self.time_crossed > self.blink_time:
                break
            
            predicted = self.predictor(image_gray, facebox)
            landmarks = self.rect_to_ndarray(predicted, 68)
            
            # eye coordinates
            right, left = landmarks[self.right_eye[0]:self.right_eye[1]], landmarks[self.left_eye[0]:self.left_eye[1]]
            # get ear for each eye
            ear_right, ear_left = self.get_ear(right), self.get_ear(left)

            if (ear_right < self.threshold or ear_left < self.threshold):
                self.crossed_right = True
                self.crossed_left = True
                self.time_crossed = time
                
            
            if (self.crossed_right == True and ear_right > self.threshold * self.prop) or (self.crossed_left == True and ear_left > self.threshold * self.prop):
                self.crossed_right = False
                self.crossed_left = False
                self.time_crossed = 0
                self.blinks += 1
            
            for eye in [right, left]:
                for (x, y) in eye:
                    cv.circle(image_predicted, (x, y), 2, RGB, -1)
                    cv.circle(image_predicted, (x, y), 2, RGB, -1)
        
        cv.putText(image_predicted, f"Blinks: {self.blinks}", (50, 50), cv.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1)
        cv.putText(image_predicted, f"RightEAR: {round(ear_right, 3)}", (350, 50), cv.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1)
        cv.putText(image_predicted, f"LeftEAR: {round(ear_left, 3)}", (350, 100), cv.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1)
        cv.putText(image_predicted, f"Threshold: {self.threshold}", (50, 400), cv.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1)
        cv.putText(image_predicted, f"Time: {time}", (350, 400), cv.FONT_HERSHEY_PLAIN, 2, (0, 0, 255), 1)
        return image_predicted

In [40]:
def show_feed(feed, title, wait=1, rewind=False):
    retval, frame_color = feed.read()
    if not retval:
        return 1 # check value did not pass

    cv.namedWindow(title, cv.WINDOW_AUTOSIZE)
    cv.setWindowProperty(title, cv.WND_PROP_VISIBLE, 1)
    time = datetime.datetime.now()
    
    while retval:
        time_delta = datetime.datetime.now() - time
        time_delta = int(time_delta.total_seconds() * 1000)
        predicted = predictor.predict(frame_color, time_delta)
        cv.imshow(title, predicted)
        
        key_input = cv.waitKey(wait)
        if key_input != -1:
            break;
        
        retval, frame_color = feed.read()
    
    try:
        if rewind: # Rewind VideoCapture object for video files
            feed.set(cv.CAP_PROP_POS_MSEC, 0.0)
        cv.destroyWindow(title)
        
    except:
        print("'" + title + "' not found")

In [97]:
predictor = PredictorBlinkClass(0.135)

In [30]:
webcam_index = 0
webcam = cv.VideoCapture(webcam_index)

In [101]:
predictor.set_threshold(0.20)
predictor.set_blinks(0)
show_feed(webcam, "Blink Detector", wait=1, rewind=True)

In [29]:
webcam.release()

In [91]:
cv.destroyWindow('Blink Detector')